# process the DNA assemblies to get viral contigs
- concatenate all contigs
- remove everything under 10kb
- rename
- run virsorter2
- run iphop
- map reads
- Run genomad for taxonomy, proteins, proviruses
- Run iphop for virus host matching
- do we want functional annotation and AMGs? --> run vibrant in that case?

All assembled contigs from the xx samples were concatenated into one fasta file. Contigs smaller than 10kb were removed, and contigs were renamed using bbmap (ref) reformat.sh and rename.sh, respectively. Virsorter2 (ref) was then ran on all remaining contigs (n=454,868), using --min-length 10000 and --min-score 0.9. Resulting viral contigs were dereplicated at approximately species level (95% ANI), over 85% of the length of the shorter contig with CD-Hit (ref), using the -c 0.95 and -aS 0.85. Clustering resulted in 44,824 vOTUs that were used for further analysis.
Bowtie2 (ref), was used to map reads to vOTUs. A bowtie2 index file was created using the bowtie2-build command, otherwise standard settings. Both metatranscriptomic reads and metagenomic reads were mapped, to get an overview of the respective 'active' viral community and the present viral community. Reads were mapped using the --no-unal and --sensitive settings. 



In [ ]:
# concatenate all available contigs
cat /home/hfm/ER_rumenShotgun/ER_Nanuq/MAG/assembled_reads/NS*/final.contigs.fa > all_DNA_contigs.Hugo.fa

# srun 
srun --account=ctbrowngrp -p bmm -J vs2 -t 0:10:00 -c 1 --mem=50gb --pty bash

# rename and remove < 10k
mamba activate bbmap
reformat.sh in=all_DNA_contigs.Hugo.fa out=all_DNA_contigs.10k.fa minlength=10000 -Xmx50g
rename.sh in=all_DNA_contigs.10k.fa out=all_DNA_contigs.10k.rn.fa prefix=ath_rumen_2024_ -Xmx50g

# Partition it because virsorter will take forever on 454,868 sequences
partition.sh in=../all_DNA_contigs.10k.rn.fa out=out_%.fasta ways=40 -Xmx50g


In [ ]:
# snakemake virsorter on the subsetted files
srun --account=ctbrowngrp -p high2 -J bt2 -t 14:00:00 -c 64 --mem=80gb --pty bash

mamba activate branchwater
snakemake --use-conda --resources mem_mb=120000 --rerun-triggers mtime \
-c 64 --rerun-incomplete -k -s Snakefile_bowtie -n


# snakemake file
FASTA, = glob_wildcards('../resources/split_DNA_contigs/{fasta}.fasta')

rule all:
    input:
        expand("../results/virsorter2/DNA/check/{fasta}.done", fasta=FASTA),

rule virsorter2:
    input:
        fasta = '../resources/split_DNA_contigs/{fasta}.fasta',
    output:
        check='../results/virsorter2/DNA/check/{fasta}.done',
    conda: 
        "virsorter2"
    threads: 16
    shell:
        """
        virsorter run all -w ../results/virsorter2/DNA/{wildcards.fasta} \
        -i {input.fasta} -d /group/jbemersogrp/databases/virsorter/ \
        --min-length 10000 -j {threads} --min-score 0.9
        """


In [ ]:
# resulted in 58,188 viral contigs
# Now concatenate and drep at 95

# srun 
srun --account=ctbrowngrp -p bmm -J vs2 -t 0:10:00 -c 1 --mem=50gb --pty bash

# rename again bc of stupid ||
mamba activate bbmap
rename.sh in=viral_contigs.fa out=viral_contigs.rn.fa prefix=ath_rumvir_24_ -Xmx50g

# split contigs for drep
mkdir contigs
cd ./contigs
awk '/^>/ {OUT=substr($0,2) ".fa"}; OUT {print >OUT}' ../viral_contigs.95.cluster.fa 

srun --account=ctbrowngrp -p bmh -J drep -t 10:00:00 -c 24 --mem=100gb --pty bash
mamba activate drep

# Run dRep at 95% ANI over 85% of length of longest contigs
dRep dereplicate ./drep --S_algorithm ANImf --ignoreGenomeQuality -l 10000 -sa 0.95 -nc 0.85 -p 24

In [ ]:
# now cdhit to deduplicate, drep being shitty (result in 44824 vir clusters)
srun --account=ctbrowngrp -p bmm -J cdhit -t 1:00:00 -c 40 --mem=70gb --pty bash

mamba activate cdhit
cd-hit-est -i viral_contigs.rn.fa \
-o viral_contigs.95.cluster.fa -d 0 \
-c 0.95 -aS 0.85 -M 95000 -T 40

In [ ]:
# bowtie2 build the db
mamba activate bowtie2
bowtie2-build ../../drep/DNA/viral_contigs.95.cluster.fa ./viral_contigs.95.cluster -p 40

In [ ]:
# do the same for the RVD
srun --account=ctbrowngrp -p bmm -J bowtie2 -t 8:00:00 -c 40 --mem=70gb --pty bash
mamba activate bowtie2
bowtie2-build ../../resources/RVD/RVD_rn.fa ./RVD -p 40 --large-index

In [ ]:
# COVERAGE TABLE
# Use coverM to create a coverage table for vOTUs
srun --account=ctbrowngrp -p bmm -J coverm -t 1:00:00 -c 1 --mem=30gb --pty bash

mamba activate coverm

# make a coverage table, where the min amount of the contig that has to be covered is 75%
coverm contig -m mean --min-covered-fraction 0.75 \
-b *.bam > ../../240923_


In [ ]:
# DNA reads:
/home/hfm/ER_rumenShotgun/ER_Nanuq/TRIMMED/*R1_trim.fastq.gz

# metaT reads
/home/hfm/Rumen_Microbiome_Genomics/1_Sequences_Guanhui/TRIMMED/18048XR-81-24_S11_L003_R2_001_trim.fixed.gz

In [ ]:
# For normalization of the coveragbin.12741e table, we need the number of reads per sample
mamba activate seqkit
seqkit stats /home/hfm/ER_rumenShotgun/ER_Nanuq/TRIMMED/*R1_trim.fastq.gz > num_reads.tsv

In [ ]:
#GENOMAD
# srun it, needs quite some mem
srun --account=ctbrowngrp -p med2 -J genomad -t 12:00:00 -c 36 --mem=100gb --pty bash

# end to end for everything, need to annotate for classification
# Use for taxonomy of DNA phage

mamba activate genomad
genomad end-to-end \
../drep/DNA/viral_contigs.95.cluster.fa \
./DNA/ ./genomad_db \
--threads 36 --enable-score-calibration \
--splits 20 --cleanup 

In [ ]:
# Iphop (maybe, also try diff method for virus-host linkages)
# Wait with running iphop until the MAGs are here?
ln -s /group/jbemersogrp/databases/iphop . 

# run it
srun --account=ctbrowngrp -p med2 -J iphop -t 24:00:00 -c 60 --mem=70gb --pty bash

mamba activate iphop_env
iphop predict -f viral_contigs.95.cluster.fa \
-o ./ -d ./iphop/latest/Aug_2023_pub_rw -t 60

In [ ]:
# Run vibrant for AMG purposes?
# not needed if running DRAM-v
srun --account=ctbrowngrp -p med2 -J vibrant -t 24:00:00 -c 40 --mem=70gb --pty bash

mamba activate vibrant
VIBRANT_run.py \
-i viral_contigs.95.cluster.fa -folder ../../VIBRANT \
-t 40 -d /group/jbemersogrp/databases/vibrant/databases

In [ ]:
# DRAM-v for AMGs as well --> do need dramV
# First need to prepare the virsorter output
srun --account=ctbrowngrp -p med2 -J VS2 -t 14:00:00 -c 80 --mem=60gb --pty bash

virsorter run --prep-for-dramv -w ../../DRAMv \
-i viral_contigs.95.cluster.fa -j 48 all \
-d /group/jbemersogrp/databases/virsorter/ 


# Is 64 hours with 80 threads not enough? :(
srun --account=ctbrowngrp -p med2 -J DRAMv -t 24:00:00 -c 40 --mem=60gb --pty bash
srun --account=ctbrowngrp -p med2 -J DRAMv -t 120:00:00 -c 80 --mem=80gb --pty bash

# then run dramv
mamba activate DRAM

/group/jbemersogrp/databases/dram/

DRAM-v.py annotate -i VS2/for-dramv/final-viral-combined-for-dramv.fa \
-v VS2/for-dramv/viral-affi-contigs-for-dramv.tab --threads 80 \
-o DRAMv --skip_trnascan

# I dont need tRNAs.

## TerL tree
- Get all TerL sequences from VIBRANT predictions (K06909) (seqtk)
- Get all TerL sequences from Refseq
- Cluster sequences at 95% AAI (CD-hit)
- remove everything < 100 AA
- align protein sequences (MAFFT)
- TrimAL
- Prottest
- IQ tree or fasttree


### Numbers:
Refseq: 4711
own: 5105
RVD: 10917
After dereplicating:
Refseq: 1830
own: 2975
RVD: 6968


In [ ]:
# go to folder
cd results/TerL_tree/

# link refseq seqs to folder
ln -s ../../resources/TerL_refseq.fasta .

# rm header spaces
cut -d ' ' -f1 prodigal_w_spaces.fa > rumvir_vibrant.faa

# Subset all protein predictions to only protein predictions that are TerL from own seqs
seqtk subseq rumvir_vibrant.faa terl.vibrant.txt > TerL.vibrant.faa

In [ ]:
# Cluster using CD-hit
srun --account=ctbrowngrp -p med2 -J cdhit -t 4:00:00 -c 15 --mem=50gb --pty bash
mamba activate cdhit

# own 
mamba activate cdhit
cd-hit -i TerL.vibrant.faa -o ../TerL.vibrant.95.cluster.faa \
-c 0.95 -T 15 -d 0

# refseq
cd-hit -i TerL_refseq.fasta \
-o ../TerL.refseq.95.cluster.fa -d 0 \
-c 0.95 -T 15


In [ ]:
# MAFFT seems to perform better
srun --account=ctbrowngrp -p med2 -J ginsi -t 4:00:00 -c 25 --mem=120gb --pty bash
mamba activate mafft

# use very fast method
mafft --retree 1 --maxiterate 0 TerL.rs.own.faa > TerL.rs.own.mafft

# also for the refseq, own, rvd
mafft --retree 1 --maxiterate 0 TerL.rs.own.rvd.faa > TerL.rs.own.rvd.mafft

# sliced out positions with >50% gaps, using trimal
# https://www.nature.com/articles/s41467-023-41075-2#Sec10
mamba activate trimal
trimal -in TerL.rs.own.mafft -gt 0.5 -out TerL.rs.own.trimal.faa
trimal -in TerL.rs.own.rvd.mafft -gt 0.5 -out TerL.rs.own.rvd.trimal.faa

# FastTree for building trees
mamba activate fasttree
FastTree < TerL.rs.own.trimal.faa > TerL.rs.own.mafft.tree -wag -mlacc 2 -slownni -log mafft.tree.log
FastTree < TerL.rs.own.rvd.trimal.faa > TerL.rs.own.rvd.mafft.tree -wag -mlacc 2 -slownni -log mafft.tree.log


In [ ]:
# Add the TerL sequences
cat RVD_*/RVD*.faa > RVD_all.faa

# grep terL (n=10917)
grep -i -e 'terminase' RVD_all.faa > TerL.RVD.headers.txt


# rm header spaces 
cut -d ' ' -f1 RVD_all.faa > RVD_all.ns.faa

# Subset all protein predictions to only protein predictions that are TerL from own seqs
seqtk subseq RVD_all.ns.faa TerL.rvd.txt > TerL.RVD.faa

In [ ]:
# derep using cdhit
srun --account=ctbrowngrp -p med2 -J ginsi -t 1:00:00 -c 15 --mem=120gb --pty bash

# own 
mamba activate cdhit
cd-hit -i TerL.RVD.faa -o TerL.RVD.95cluster.faa \
-c 0.95 -T 15 -d 0

In [ ]:
# # Align with clustalo
# mamba activate clustalo
# clustalo -i TerL.vibrant.refseq.faa \
# --auto -t Protein --threads=15 \
# -o TerL.refseq.alignment


# # also try ginsi
# mafft --globalpair --maxiterate 1000 \
# --thread 25 TerL.vibrant.refseq.faa > TerL.refseq.mafft.ginsi

In [ ]:
# Mapping to RVD also gained 40.000 vOTUs. Now drep with own vOTUs
# 44824 own vOTUs, 49066 RVD
filterbyname.sh in=../DNA/viral_contigs.95.cluster.fa out=own.fa names=contigs.txt include=t
filterbyname.sh in=../../../resources/RVD/RVD_rn.fa \
out=RVD.fa names=contigs.txt include=t


# now cdhit to deduplicate, drep being shitty (result in 44824 vir clusters)
srun --account=ctbrowngrp -p bmm -J cdhit -t 4:00:00 -c 40 --mem=90gb --pty bash

# sort by length first
sortbyname.sh in=ownRVD.fa out=ownRVD.sortlen.fa descending

# result is 91,461 vOTUs
mamba activate cdhit
cd-hit-est -i ownRVD.sortlen.fa \
-o ownRVD.95.cluster.fa -d 0 \
-c 0.95 -aS 0.85 -M 90000 -T 40

In [ ]:
# Map reads to this dataset?
# Probably should remove the RVD mapping only in that case. 
# bowtie2 build the db
srun --account=ctbrowngrp -p med2 -J bt2 -t 10:00:00 -c 40 --mem=80gb --pty bash

mamba activate bowtie2
bowtie2-build ../drep/RVD_own/ownRVD.95.cluster.fa \
./ownRVD.95.cluster.bowtie -p 40 --large-index

mamba activate branchwater
snakemake --use-conda --resources mem_mb=80000 --rerun-triggers mtime \
-c 40 --rerun-incomplete -k -s Snakefile_bowtie -n